# Initiation

In [42]:
import boto3
import json
import os
import sys
import time
sys.path.append("../")

from dotenv import load_dotenv
from utils.openaiClient import OpenAIClient
from textractClient import (
    start_job,
    is_job_complete, 
    get_job_results,
    save_results_to_file,
    load_textract_pages,
    extract_tables_from_blocks_unmerged,
    extract_forms_kv,
    write_tables_csv,
    write_forms_csv,
    build_llm_jsonl
)
import importlib

import textractClient
importlib.reload(textractClient)
from textractClient import *



# Load the .env file (assuming it’s in the same folder)
load_dotenv()
AWS_REGION = os.getenv("AWS_REGION")

boto3.setup_default_session(
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=AWS_REGION
)

openai_client = OpenAIClient(api_key=os.getenv("OPENAI_API_KEY"))

# Extract from document

In [43]:
CATALOG_NAME = 'ms-02-02'
CATALOG_FILE = f'{CATALOG_NAME}.pdf'

S3_BUCKET = 'catalogs-for-extraction'
S3_CATALOG_PATH = f's3://{S3_BUCKET}/{CATALOG_FILE}'

TEXTRACT_OUTPUT_JSON_PATH = f"textract_output_{CATALOG_NAME}.json"

# job_id = start_job(s3_bucket, document, features=['TABLES'], region=AWS_REGION)
job_id = "bce4b0bc30439528968cc52ca5ae225868056799096afbcc6944bdaef4fe81df"

In [ ]:
status = is_job_complete(job_id, region=AWS_REGION)
results_pages = get_job_results(job_id, region=AWS_REGION)
save_results_to_file(results_pages, TEXTRACT_OUTPUT_JSON_PATH)

Job status: SUCCEEDED
Retrieved 1000 blocks on this page.
Fetching next page of results...
Retrieved 733 blocks on this page.
Saved results to textract_output_ms-02-02.json


# Parse table

In [44]:
pages = load_textract_pages(TEXTRACT_OUTPUT_JSON_PATH)

# Flatten all Blocks across responses (Textract paginates API, not your PDF pages)
all_blocks = []
for part in pages:
    all_blocks.extend(part.get("Blocks", []))

tables = extract_tables_from_blocks_unmerged(all_blocks, replicate_data=True, header_scan_rows=2)

In [45]:
table = tables[2]
table.keys()

dict_keys(['page', 'table_index', 'headers', 'rows', 'spans'])

In [13]:
for header in table['headers']:
    print(header)


Main Line Tubing Size / in.
Branch Swagelok Tube / Fitting Size
Orifice / in. (mm)
Ordering / Number
Dimensions, / A
in. (mm) / B


In [46]:
for row in table['rows']:
    print(row)

['4 { 0.083', '2 in.', '1.500 (38.1)', 'SS-D68TTE16S32', '23.0 (584)', '17.2 (437)']
['4 { 0.083', '1 1/2 in.', '1.250 (31.8)', 'SS-D67TTE16S24', '23.0 (584)', '14.4 (365)']
['2 { 0.065', '3/4 in.', '0.516 (13.1)', 'SS-D63TTE8S12', '13.0 (330)', '8.19 (208)']
['2 { 0.065', '1/2 in.', '0.411 (10.4)', 'SS-D63TTE8S8', '13.0 (330)', '8.19 (208)']
['1 1/2 { 0.065', '3/4 in.', '0.516 (13.1)', 'SS-D63TTE6S12', '13.0 (330)', '7.13 (181)']
['1 1/2 { 0.065', '1/2 in.', '0.411 (10.4)', 'SS-D63TTE6S8', '13.0 (330)', '7.13 (181)']
['1 1/2 { 0.065', '3/8 in.', '0.281 (7.1)', 'SS-D63TTE6S6', '13.0 (330)', '6.94 (176)']
['1 { 0.065', '3/4 in.', '0.516 (13.1)', 'SS-D63TTE4S12', '13.0 (330)', '6.88 (175)']
['1 { 0.065', '1/2 in.', '0.411 (10.4)', 'SS-D63TTE4S8', '13.0 (330)', '6.88 (175)']
['1 { 0.065', '3/8 in.', '0.281 (7.1)', 'SS-D63TTE4S6', '13.0 (330)', '6.69 (170)']
['3/4 { 0.065', '3/8 in.', '0.281 (7.1)', 'SS-D62TTE3S6', '9.00 (229)', '5.19 (132)']
['3/4 { 0.065', '1/4 in.', '0.187 (4.8)', 'SS-D

In [ ]:
def _collect_cells_for_table(tblock, id_map):
    """Return list of CELLs, and MERGED_CELLs (if any) for this table."""
    cells, merged = [], []
    for rel in tblock.get("Relationships", []):
        if rel.get("Type") != "CHILD":
            continue
        for cid in rel.get("Ids", []):
            b = id_map.get(cid)
            if not b: 
                continue
            bt = b["BlockType"]
            if bt == "CELL":
                cells.append(b)
            elif bt == "MERGED_CELL":
                merged.append(b)
    return cells, merged

In [34]:
for rel in tblock.get("Relationships", []):
    print(rel)
id_map.get('7b78171d-5a04-4949-9759-aa37097d725d')['BlockType']

{'Type': 'CHILD', 'Ids': ['c66800ec-f19e-410c-985e-dcc992436f15', '1787e428-e78b-45d2-b984-3ccbbcec4424', 'f22be46a-7d1a-424a-a12b-c92c9313dc0f', '4b02e913-02d2-4bfe-a653-b35baed2d537', '8fd30a9d-ec2e-4f7c-be0c-235da64105f1', '1c37c218-e946-43f0-b65f-9fdd49e45e07', '64f7f690-a8a1-4b7d-ade4-adad9f157c28', '7b1dc9a5-75f4-4754-ace0-f836499d693c', '0e76c060-d2b7-4acf-bf15-a71f94796b5d', '52e5df33-fcd3-4ddc-a625-557e8a3b8950', '42823aa2-af67-446f-b212-8b401a7101fb', '9ac5e39e-3288-4255-9cb7-7183eca3a6c8', 'ed9c4644-7623-4d9a-bce4-d3c402288925', 'ec432e43-5ae2-49a5-9356-dbe3d3d4a931', 'e5a0a18b-9898-49c7-bc0e-bbdbec98766e', '6457153d-f8ab-4ccd-af42-1fe6d6d78112', '01124359-e238-4591-8664-34b023e64998', '32e70555-3ab3-44ab-bdc9-18d4a7c555be', 'df96cb0b-c692-4ac5-bc36-3909696489e4', 'fc6818eb-df6b-41fe-acdb-6a7542d44047', '921b34ef-bb4e-4661-9e81-2a44829f016f', 'd02464ad-3ad7-4ba5-98f4-9be6493bbc83', '25777f95-c963-4e29-be28-f4f6068045ec', '20875896-a42d-44cb-811b-03d808614d4f', '6a302d80-d6a6

'MERGED_CELL'

In [41]:

blocks = all_blocks
replicate_data = True
header_scan_rows = 2
id_map, type_map = build_block_maps(blocks)

out = []
# for t_index, tblock in enumerate(type_map.get("TABLE", []), start=1):
tblock = type_map.get("TABLE", [])[2]
page = tblock.get("Page", None)

cells, merged_cells = _collect_cells_for_table(tblock, id_map)

# Compute table size considering spans
max_row = max((c.get("RowIndex", 0) + c.get("RowSpan", 1) - 1) for c in cells) if cells else 0
max_col = max((c.get("ColumnIndex", 0) + c.get("ColumnSpan", 1) - 1) for c in cells) if cells else 0

print(max_row, max_col)

# Init grid + span holders
grid = [["" for _ in range(max_col)] for _ in range(max_row)]
spans = [[(1,1) for _ in range(max_col)] for _ in range(max_row)]
is_header_row = [False]*max_row  # we'll guess header band later

# Step 1: Process MERGED_CELLs first to establish merged regions and find their text
# Build a map of merged regions: (r0, c0, rs, cs) -> text_value
merged_regions = {}  # key: (r0, c0, rs, cs), value: text

for m in merged_cells:
    print("merged cell", m)
    # Textract links MERGED_CELL -> CHILD -> CELL ids
    child_ids = []
    for rel in m.get("Relationships", []):
        if rel.get("Type") == "CHILD":
            child_ids.extend(rel.get("Ids", []))
    child_cells = [id_map[i] for i in child_ids if id_map.get(i) and id_map[i].get("BlockType")=="CELL"]
    if not child_cells:
        continue
    
    # Compute merged area boundaries
    r_indices = []
    c_indices = []
    for cc in child_cells:
        ri = cc.get("RowIndex", 1)-1
        ci = cc.get("ColumnIndex", 1)-1
        rs = cc.get("RowSpan", 1)
        cs = cc.get("ColumnSpan", 1)
        r_indices.extend(list(range(ri, ri+rs)))
        c_indices.extend(list(range(ci, ci+cs)))
    
    r0, c0 = min(r_indices), min(c_indices)
    rs = max(r_indices) - r0 + 1
    cs = max(c_indices) - c0 + 1
    
    # Try to get text from MERGED_CELL block first, then from any child cell
    txt = get_text_for_block(m, id_map)
    if not txt or not txt.strip():
        # Search through all child cells to find where the text actually is
        for cc in child_cells:
            txt = get_text_for_block(cc, id_map)
            if txt and txt.strip():
                break
    
    if txt and txt.strip():
        merged_regions[(r0, c0, rs, cs)] = txt.strip()

# Step 2: Process all CELLs and place their text
# Also track which cells are part of merged regions
cell_spans = {}  # key: (r0, c0), value: (rs, cs)

for c in cells:
    r0 = c.get("RowIndex", 1)-1
    c0 = c.get("ColumnIndex", 1)-1
    rs = c.get("RowSpan", 1)
    cs = c.get("ColumnSpan", 1)
    
    # Store span information
    cell_spans[(r0, c0)] = (rs, cs)
    spans[r0][c0] = (rs, cs)
    
    # Get text for this cell
    txt = get_text_for_block(c, id_map)
    if txt and txt.strip():
        grid[r0][c0] = txt.strip()

# Step 3: Apply merged region text to all positions in the merged area
for (r0, c0, rs, cs), txt in merged_regions.items():
    for rr in range(r0, r0+rs):
        for cc in range(c0, c0+cs):
            grid[rr][cc] = txt
            spans[rr][cc] = (rs, cs)

# Step 4: If replicate_data=True, replicate text across all spans
# For each cell with a span, collect text from ALL cells within that span, join them, and replicate
span_regions = {}  # key: (r0, c0, rs, cs), value: set of (r, c) positions

if replicate_data:
    # Create a map of span regions: (r0, c0, rs, cs) -> set of all cells in that span
    
    # Process regular cells with spans
    for (r0, c0), (rs, cs) in cell_spans.items():
        if rs > 1 or cs > 1:  # Only process cells that actually span
            # Collect all positions in this span
            positions = set()
            for rr in range(r0, r0+rs):
                for cc in range(c0, c0+cs):
                    positions.add((rr, cc))
            span_regions[(r0, c0, rs, cs)] = positions
    
    # Process merged regions
    for (r0, c0, rs, cs) in merged_regions.keys():
        positions = set()
        for rr in range(r0, r0+rs):
            for cc in range(c0, c0+cs):
                positions.add((rr, cc))
        span_regions[(r0, c0, rs, cs)] = positions
    
    # For each span region, collect text from ALL cells within it, join, and replicate
    for (r0, c0, rs, cs), positions in span_regions.items():
        # Collect all text values from cells within this span
        text_parts = []
        for (rr, cc) in positions:
            cell_text = grid[rr][cc]
            if cell_text and cell_text.strip():
                text_parts.append(cell_text.strip())
        
        # Join all text parts with space
        if text_parts:
            joined_text = " ".join(text_parts)
            # Replicate the joined text to ALL cells in the span
            for (rr, cc) in positions:
                grid[rr][cc] = joined_text

# ---- Header detection & composition (multi-row headers) ----
# Density-based header band guess
densities = [sum(1 for x in row if x.strip()) for row in grid[:min(header_scan_rows, max_row)]]
header_end = 0
for i in range(len(densities)):
    header_end = i
    if i > 0 and densities[i] <= max(1, densities[i-1]//2):
        break
header_rows = list(range(0, header_end+1))
for r in header_rows:
    is_header_row[r] = True

# Compose headers: parent spans replicated above child columns become "Parent / Child"
headers = []
for c in range(max_col):
    parts = []
    for r in header_rows:
        t = grid[r][c].strip()
        if t and (not parts or t.lower() != parts[-1].lower()):
            parts.append(t)
    header = " / ".join(parts) if parts else f"col_{c+1}"
    headers.append(header)

# Step 5: Handle cells that should be merged but Textract didn't detect as merged
# This is a fallback for cases where Textract doesn't detect RowSpan > 1
# We infer merged groups by looking for patterns: non-empty cell followed by empty cells until next non-empty
# But only do this if replicate_data=True and we haven't already handled these cells via spans
# Also: skip header region when inferring (use header_scan_rows as a hint, but don't rely on exact detection)
if replicate_data:
    # Track which positions are already covered by explicit spans
    covered_positions = set()
    for (r0, c0, rs, cs) in span_regions.keys():
        for rr in range(r0, r0+rs):
            for cc in range(c0, c0+cs):
                covered_positions.add((rr, cc))
    
    # Start inference from after the header region (use header_scan_rows as hint)
    data_start_row = min(header_scan_rows + 1, max_row)
    
    # For each column, identify inferred merged groups (only for positions not already covered)
    for col in range(max_col):
        row = data_start_row  # Start from after header region
        while row < max_row:
            # Skip if this position is already covered by an explicit span
            if (row, col) in covered_positions:
                row += 1
                continue
            
            # Check if current cell has a value
            if grid[row][col] and grid[row][col].strip():
                # Found start of a potential merged cell group
                value = grid[row][col]
                group_start = row
                
                # Find the end of this group (next non-empty cell or end of table)
                # Also collect all text values in this group
                text_parts = [value.strip()]
                row += 1
                group_positions = [(group_start, col)]
                
                while row < max_row:
                    # Skip if this position is already covered by an explicit span
                    if (row, col) in covered_positions:
                        break
                    
                    # Stop if we hit a non-empty cell (start of next group)
                    if grid[row][col] and grid[row][col].strip():
                        break
                    
                    # This is an empty cell in the same group
                    group_positions.append((row, col))
                    row += 1
                
                # If we found a group with multiple positions, join and replicate
                if len(group_positions) > 1:
                    # Collect any additional text from cells in the group
                    for (rr, cc) in group_positions[1:]:
                        if grid[rr][cc] and grid[rr][cc].strip():
                            text_parts.append(grid[rr][cc].strip())
                    
                    # Join all text parts and replicate
                    if text_parts:
                        joined_text = " ".join(text_parts)
                        for (rr, cc) in group_positions:
                            grid[rr][cc] = joined_text
            else:
                # Empty cell, move to next
                row += 1

# Body rows
body_rows = [grid[r] for r in range(max_row) if not is_header_row[r]]

out.append({
    "page": page,
    "table_index": t_index,
    "headers": headers,
    "rows": body_rows,
    "spans": spans,  # keep for downstream logic if needed
})


out[0]

CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
CELL
16 6


{'page': 2,
 'table_index': 1,
 'headers': ['Main Line Tubing Size / in.',
  'Branch Swagelok Tube / Fitting Size',
  'Orifice / in. (mm)',
  'Ordering / Number',
  'Dimensions, / A',
  'in. (mm) / B'],
 'rows': [['', '2 in.', '1.500 (38.1)', 'SS-D68TTE16S32', '', '17.2 (437)'],
  ['4 { 0.083',
   '1 1/2 in.',
   '1.250 (31.8)',
   'SS-D67TTE16S24',
   '23.0 (584)',
   '14.4 (365)'],
  ['4 { 0.083',
   '3/4 in.',
   '0.516 (13.1)',
   'SS-D63TTE8S12',
   '23.0 (584)',
   '14.4 (365)'],
  ['2 { 0.065',
   '1/2 in.',
   '0.411 (10.4)',
   'SS-D63TTE8S8',
   '13.0 (330)',
   '8.19 (208)'],
  ['2 { 0.065',
   '3/4 in.',
   '0.516 (13.1)',
   'SS-D63TTE6S12',
   '13.0 (330)',
   '8.19 (208)'],
  ['1 1/2 { 0.065',
   '1/2 in.',
   '0.411 (10.4)',
   'SS-D63TTE6S8',
   '13.0 (330)',
   '7.13 (181)'],
  ['1 1/2 { 0.065',
   '3/8 in.',
   '0.281 (7.1)',
   'SS-D63TTE6S6',
   '13.0 (330)',
   '6.94 (176)'],
  ['1 1/2 { 0.065',
   '3/4 in.',
   '0.516 (13.1)',
   'SS-D63TTE4S12',
   '13.0 (330)',